In [ ]:
import os, cv2, torch, shutil
import numpy as np
from tqdm import tqdm
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.cluster import KMeans


assert torch.cuda.is_available(), "❌ CUDA not available! Fix GPU before running."
device = torch.device("cuda")
print("✅ Using GPU:", torch.cuda.get_device_name(0))


VIDEO_PATH = "input/videos"
FRAME_DIR = "output/frames"
FACE_DIR = "output/faces"
CLUSTER_DIR = "output/faces_clustered"
NUM_PEOPLE = 5

os.makedirs(FRAME_DIR, exist_ok=True)
os.makedirs(FACE_DIR, exist_ok=True)
os.makedirs(CLUSTER_DIR, exist_ok=True)
for i in range(NUM_PEOPLE):
    os.makedirs(os.path.join(CLUSTER_DIR, f"person_{i+1}"), exist_ok=True)


video_file = next(f for f in os.listdir(VIDEO_PATH) if f.lower().endswith((".mp4",".avi",".mov")))
video_file = os.path.join(VIDEO_PATH, video_file)
print("🎥 Video:", video_file)


cap = cv2.VideoCapture(video_file)
count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imwrite(os.path.join(FRAME_DIR, f"frame_{count:06d}.jpg"), frame)
    count += 1
cap.release()
print(f"🖼 Extracted {count} frames")


mtcnn = MTCNN(keep_all=True, device=device)
face_count = 0

for f in tqdm(sorted(os.listdir(FRAME_DIR)), desc="Detecting Faces"):
    img = cv2.imread(os.path.join(FRAME_DIR, f))
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    boxes, _ = mtcnn.detect(rgb)
    if boxes is None:
        continue
    for b in boxes:
        x1,y1,x2,y2 = map(int,b)
        face = img[y1:y2, x1:x2]
        if face.size == 0:
            continue
        cv2.imwrite(os.path.join(FACE_DIR, f"face_{face_count:06d}.jpg"), face)
        face_count += 1

print(f"🙂 Detected {face_count} faces")

assert face_count > 0, "❌ No faces detected. Check video quality or camera angle."


model = InceptionResnetV1(pretrained="vggface2").eval().to(device)
embeddings = []
face_files = sorted(os.listdir(FACE_DIR))

for f in tqdm(face_files, desc="Embedding Faces"):
    img = cv2.imread(os.path.join(FACE_DIR, f))
    img = cv2.resize(img, (160,160))
    img = img[:,:,::-1].copy()

    img = torch.tensor(img).permute(2,0,1).unsqueeze(0).float().div(255).to(device)

    with torch.no_grad():
        emb = model(img).cpu().numpy()[0]

    embeddings.append(emb)

embeddings = np.array(embeddings)


kmeans = KMeans(n_clusters=NUM_PEOPLE, random_state=42, n_init="auto")
labels = kmeans.fit_predict(embeddings)

for f, l in zip(face_files, labels):
    shutil.copy(
        os.path.join(FACE_DIR, f),
        os.path.join(CLUSTER_DIR, f"person_{l+1}", f)
    )

print("🎉 DONE — GPU pipeline complete.")


✅ Using GPU: NVIDIA GeForce RTX 4050 Laptop GPU
🎥 Video: input/videos\We all miss you guys🥹......#strangerthings #strangerthingss05 #steveherrington #netflix.mp4
🖼 Extracted 815 frames


Detecting Faces: 100%|██████████| 815/815 [01:11<00:00, 11.47it/s]


🙂 Detected 1210 faces


Embedding Faces: 100%|██████████| 1210/1210 [00:37<00:00, 32.22it/s]


🎉 DONE — GPU pipeline complete.


In [ ]:
import os, random, shutil, yaml
from glob import glob
from sklearn.model_selection import train_test_split
from ultralytics import YOLO


SOURCE_DIR = "output/faces_clustered"
DATASET_DIR = "yolo_dataset"
IMG_SIZE = 224
EPOCHS = 100
BATCH = 16
PATIENCE = 10  
for split in ["train", "val", "test"]:
    os.makedirs(f"{DATASET_DIR}/images/{split}", exist_ok=True)
    os.makedirs(f"{DATASET_DIR}/labels/{split}", exist_ok=True)


class_names = sorted(os.listdir(SOURCE_DIR))
class_map = {name: idx for idx, name in enumerate(class_names)}

image_records = []

for cls in class_names:
    files = glob(os.path.join(SOURCE_DIR, cls, "*.jpg"))
    for f in files:
        image_records.append((f, class_map[cls]))

train, temp = train_test_split(image_records, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

def save_split(records, split):
    for img_path, label in records:
        name = os.path.basename(img_path)
        shutil.copy(img_path, f"{DATASET_DIR}/images/{split}/{name}")
        with open(f"{DATASET_DIR}/labels/{split}/{name.replace('.jpg','.txt')}", "w") as f:
            f.write(f"{label} 0.5 0.5 1.0 1.0\n")

save_split(train, "train")
save_split(val, "val")
save_split(test, "test")

print("✅ Dataset created.")

data_yaml = {
    "path": os.path.abspath(DATASET_DIR),
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",
    "nc": len(class_names),
    "names": class_names
}

with open(f"{DATASET_DIR}/data.yaml", "w") as f:
    yaml.dump(data_yaml, f)

print("✅ data.yaml written.")

try:
    model = YOLO("yolo12n.pt")
    print("Using YOLOv12")
except:
    model = YOLO("yolov8n.pt")
    print("YOLOv12 not found — using YOLOv8")


results = model.train(
    data=f"{DATASET_DIR}/data.yaml",
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    device=0,
    patience=PATIENCE,
    save=True,
    save_period=1,
    project="runs",
    name="face_id_train",
    exist_ok=True
)

print("✅ Training complete.")


run_dir = model.trainer.save_dir
best_model = os.path.join(run_dir, "weights", "best.pt")
last_model = os.path.join(run_dir, "weights", "last.pt")

print("\nBest model:", best_model)
print("Last model:", last_model)

val_metrics = model.val(model=best_model)
print("\n📊 Validation Metrics:")
print(val_metrics)

test_metrics = model.val(model=best_model, split="test")
print("\n📊 Test Metrics:")
print(test_metrics)


print("\nConfusion matrix saved in:")
print(os.path.join(run_dir, "confusion_matrix.png"))

print("\n🎉 Training, validation and testing complete.")


✅ Dataset created.
✅ data.yaml written.
Using YOLOv12
New https://pypi.org/project/ultralytics/8.3.249 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.241  Python-3.10.19 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosa

c:\Users\Acer\miniconda3\envs\dlib310\lib\site-packages\ultralytics\nn\modules\block.py:1299: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ..\aten\src\ATen\Context.cpp:164.)
  attn = (q.transpose(-2, -1) @ k) * self.scale
c:\Users\Acer\miniconda3\envs\dlib310\lib\site-packages\ultralytics\nn\modules\block.py:1301: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this ope

AMP: checks passed 
train: Fast image access  (ping: 0.00.0 ms, read: 4.13.6 MB/s, size: 9.5 KB)
train: Scanning E:\face_data_pipeline\yolo_dataset\labels\train... 847 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 847/847 1.0Kit/s 0.8s0.0ss
train: New cache created: E:\face_data_pipeline\yolo_dataset\labels\train.cache
val: Fast image access  (ping: 0.10.0 ms, read: 2.82.5 MB/s, size: 9.6 KB)
val: Scanning E:\face_data_pipeline\yolo_dataset\labels\val... 181 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 181/181 841.1it/s 0.2s0.0s
val: New cache created: E:\face_data_pipeline\yolo_dataset\labels\val.cache
Plotting labels to E:\face_data_pipeline\runs\face_id_train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 113 weight(decay=0.0), 120 weight(decay=0.0005), 119 bias(decay=0.0)
Image sizes 224 t

c:\Users\Acer\miniconda3\envs\dlib310\lib\site-packages\ultralytics\nn\modules\block.py:1681: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ..\aten\src\ATen\Context.cpp:164.)
  attn = (q.transpose(-2, -1) @ k) * (self.head_dim**-0.5)
c:\Users\Acer\miniconda3\envs\dlib310\lib\site-packages\ultralytics\nn\modules\block.py:1683: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, b

      1/100     0.811G     0.6524      3.489      1.198         44        224: 0% ──────────── 0/53  2.5s

...

     54/100     0.838G    0.08685     0.2765     0.8814         50        224: 100% ━━━━━━━━━━━━ 53/53 10.7it/s 5.0s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 10.4it/s 0.6s.3s
                   all        181        181      0.982      0.961      0.992      0.991

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     55/100     0.838G    0.08958     0.2974      0.881         49        224: 2% ──────────── 1/53 1.7it/s 0.2s<30.9s

c:\Users\Acer\miniconda3\envs\dlib310\lib\site-packages\ultralytics\nn\modules\block.py:1681: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ..\aten\src\ATen\Context.cpp:164.)
  attn = (q.transpose(-2, -1) @ k) * (self.head_dim**-0.5)
c:\Users\Acer\miniconda3\envs\dlib310\lib\site-packages\ultralytics\nn\modules\block.py:1683: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, b

     55/100     0.838G    0.08852     0.2802     0.8715         40        224: 100% ━━━━━━━━━━━━ 53/53 10.8it/s 4.9s0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 10.9it/s 0.6s.4s
                   all        181        181      0.976      0.968       0.99      0.989

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


c:\Users\Acer\miniconda3\envs\dlib310\lib\site-packages\ultralytics\nn\modules\block.py:1681: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility (Triggered internally at ..\aten\src\ATen\Context.cpp:164.)
  attn = (q.transpose(-2, -1) @ k) * (self.head_dim**-0.5)
c:\Users\Acer\miniconda3\envs\dlib310\lib\site-packages\ultralytics\nn\modules\block.py:1683: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, b

     56/100     0.838G    0.08763     0.2735     0.8733         47        224: 100% ━━━━━━━━━━━━ 53/53 7.4it/s 7.1s0.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 4.4it/s 1.4s0.3s
                   all        181        181      0.979      0.978      0.992      0.992
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 46, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

56 epochs completed in 0.176 hours.
Optimizer stripped from E:\face_data_pipeline\runs\face_id_train\weights\last.pt, 5.5MB
Optimizer stripped from E:\face_data_pipeline\runs\face_id_train\weights\best.pt, 5.5MB

Validating E:\face_data_pipeline\runs\face_id_train\weights\best.pt...
Ultralytics 8.3.241  Python-3.10.19 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 

In [2]:
import os, cv2, torch, numpy as np
from facenet_pytorch import InceptionResnetV1

device = "cuda" if torch.cuda.is_available() else "cpu"
recognizer = InceptionResnetV1(pretrained="vggface2").eval().to(device)

DB_DIR = "output/faces_clustered"
face_db = {}

for person in os.listdir(DB_DIR):
    embs = []
    for img_name in os.listdir(os.path.join(DB_DIR, person)):
        img = cv2.imread(os.path.join(DB_DIR, person, img_name))
        img = cv2.resize(img,(160,160))
        img = img[:,:,::-1].copy()
        img_t = torch.tensor(img).permute(2,0,1).unsqueeze(0).float().div(255).to(device)
        with torch.no_grad():
            emb = recognizer(img_t).cpu().numpy()[0]
        emb = emb/np.linalg.norm(emb)
        embs.append(emb)

    face_db[person] = np.mean(embs, axis=0)
    face_db[person] /= np.linalg.norm(face_db[person])

np.save("face_db.npy", face_db)
print("Rebuilt face_db after removing person_1")


Rebuilt face_db after removing person_1


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from facenet_pytorch import MTCNN, InceptionResnetV1
import cv2, numpy as np, torch


device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


detector = MTCNN(keep_all=True, device=device)
recognizer = InceptionResnetV1(pretrained="vggface2").eval().to(device)
face_db = np.load("face_db.npy", allow_pickle=True).item()

print("Loaded identities:", list(face_db.keys()))

uploader = widgets.FileUpload(accept="image/*", multiple=False)
output = widgets.Output()
display(widgets.VBox([uploader, output]))

THRESHOLD = 0.6

def cosine(a,b):
    return np.dot(a,b)

def on_upload(change):
    with output:
        output.clear_output()
        if not uploader.value:
            return

        file_info = list(uploader.value.values())[0] if isinstance(uploader.value, dict) else uploader.value[0]
        img = cv2.imdecode(np.frombuffer(file_info['content'], np.uint8), cv2.IMREAD_COLOR)
        rgb = img[:,:,::-1]

        boxes, _ = detector.detect(rgb)
        if boxes is None:
            print("❌ No face detected")
            return

        areas = (boxes[:,2]-boxes[:,0])*(boxes[:,3]-boxes[:,1])
        x1,y1,x2,y2 = boxes[areas.argmax()].astype(int)

        face = img[y1:y2, x1:x2]
        face = cv2.resize(face,(160,160))
        face = face[:,:,::-1].copy()

        face_t = torch.tensor(face).permute(2,0,1).unsqueeze(0).float().div(255).to(device)

        with torch.no_grad():
            emb = recognizer(face_t).cpu().numpy()[0]
        emb = emb / np.linalg.norm(emb)

        scores = {k: cosine(emb,v) for k,v in face_db.items()}
        best_person, best_score = max(scores.items(), key=lambda x: x[1])

        print("Similarity:", {k: round(v,2) for k,v in scores.items()})

        if best_score < THRESHOLD:
            print(f"❌ Unknown (score={best_score:.2f})")
            return

        print(f"✅ Recognized: {best_person} (score={best_score:.2f})")
        cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
        cv2.putText(img,best_person,(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0),2)

        display(widgets.Image(value=cv2.imencode(".png",img)[1].tobytes()))

uploader.observe(on_upload, names="value")
